In [4]:
from tsai.imports import *
from tsai.models.layers import *
from torchinfo import summary
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from scipy import signal

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [28]:
num_class = 12
close_num = 12
batch_size = 64
learning_rate = 0.001
num_epochs = 100
sequence_len = 1000 * 5 # sampling_rate * second
overlap = int(sequence_len * 0.3)

In [32]:
data_list = []
scaler = MinMaxScaler((0, 1))

for i in range(1, num_class + 1):
    file_path_fil = "./data/BPF/filterd%02d.csv" % i
    file_path_unfil = "./data/BPF/unfilterd%02d.csv" % i
    data_fil = pd.read_csv(file_path_fil)
    data_np_fil = data_fil.to_numpy().flatten()
    # df_fil = pd.DataFrame(data_fil)
    # data_unfil = pd.read_csv(file_path_unfil)
    # df_unfil = pd.DataFrame(data_unfil)

    end = len(data_np_fil)
    n = 0
    n_stop = sequence_len
    data_segs = []
    while n_stop < end:
        n_start = 0 + ((sequence_len - 1) - (overlap - 1)) * n
        n_stop = n_start + sequence_len
        tmp = []
        seg = data_np_fil[n_start:n_stop].copy()
        data_segs.append([seg])
        n += 1

    data_list.append(data_segs)

In [34]:
data_list[4][0]

[array([0.37779185, 0.37960583, 0.38139324, ..., 0.00149465, 0.00150279,
        0.00151537])]

In [29]:
labels = []
for i in range(len(data_list)):
    for j in range(len(data_list[i])):
        if i <= close_num:
            labels.append(i)
        else:
            labels.append(close_num + 1)

In [58]:
data_df_list = pd.DataFrame(data_list)
tmp = data_df_list.to_numpy().flatten().copy()
data_series_list = pd.Series(tmp).dropna()
data_np_list = data_series_list.to_numpy().flatten() # tmp => data_np_list
labels_np = np.array(labels)
for i in reversed(range(len(data_np_list))):
    if len(data_np_list[i][0]) != sequence_len: # 決まった長さでないといけない
        data_np_list = np.delete(data_np_list, i)
        labels_np = np.delete(labels_np, i)
data_series_list = pd.Series(data_np_list)
labels_series = pd.Series(labels_np)
data_tensor_list = torch.tensor(data_series_list)
labels_tensor = torch.tensor(labels_series)
data_tensor_list.shape, labels_tensor.shape

(torch.Size([3657, 1, 5000]), torch.Size([3657]))

In [59]:
class InceptionModule(Module):
    def __init__(self, ni, nf, ks=40, bottleneck=True):
        ks = [ks // (2**i) for i in range(3)]
        ks = [k if k % 2 != 0 else k - 1 for k in ks]  # ensure odd ks
        bottleneck = bottleneck if ni > 1 else False
        self.bottleneck = Conv1d(ni, nf, 1, bias=False) if bottleneck else noop
        self.convs = nn.ModuleList([Conv1d(nf if bottleneck else ni, nf, k, bias=False) for k in ks])
        self.maxconvpool = nn.Sequential(*[nn.MaxPool1d(3, stride=1, padding=1), Conv1d(ni, nf, 1, bias=False)])
        self.concat = Concat()
        self.bn = BN1d(nf * 4)
        self.act = nn.ReLU()

    def forward(self, x):
        input_tensor = x
        x = self.bottleneck(input_tensor)
        x = self.concat([l(x) for l in self.convs] + [self.maxconvpool(input_tensor)])
        return self.act(self.bn(x))


@delegates(InceptionModule.__init__)
class InceptionBlock(Module):
    def __init__(self, ni, nf=32, residual=True, depth=6, **kwargs):
        self.residual, self.depth = residual, depth
        self.inception, self.shortcut = nn.ModuleList(), nn.ModuleList()
        for d in range(depth):
            self.inception.append(InceptionModule(ni if d == 0 else nf * 4, nf, **kwargs))
            if self.residual and d % 3 == 2: 
                n_in, n_out = ni if d == 2 else nf * 4, nf * 4
                self.shortcut.append(BN1d(n_in) if n_in == n_out else ConvBlock(n_in, n_out, 1, act=None))
        self.add = Add()
        self.act = nn.ReLU()
        
    def forward(self, x):
        res = x
        for d, l in enumerate(range(self.depth)):
            x = self.inception[d](x)
            if self.residual and d % 3 == 2: res = x = self.act(self.add(x, self.shortcut[d//3](res)))
        return x

    
@delegates(InceptionModule.__init__)
class InceptionTime(Module):
    def __init__(self, c_in, c_out, seq_len=None, nf=32, nb_filters=None, **kwargs):
        nf = ifnone(nf, nb_filters) # for compatibility
        self.inceptionblock = InceptionBlock(c_in, nf, **kwargs) # c_in is input channel num of conv1d
        self.gap = GAP1d(1)
        self.fc = nn.Linear(nf * 4, c_out) # c_out is 1d output size 
        self.fc_tsne = nn.Linear(nf * 4, 2)
        self.two_vecs_train = [] # list is faster in appending
        self.two_vecs_test = []

    def forward(self, x):
        x = self.inceptionblock(x)
        x = self.gap(x)
        two_dimensional_vec = self.fc_tsne(x)
        if self.training:
            self.two_vecs_train.append(two_dimensional_vec.tolist()) # あとでネストしたものをまとめてtensorかndarrayに変換するため
        else:
            self.two_vecs_test.append(two_dimensional_vec.tolist())
        x = self.fc(x)
        return x

In [62]:
class HCU_Dataset(Dataset):
    def __init__(self, dataset, labels) -> None:
        # super().__init__()
        self.radar_heartbeat = dataset
        self.labels = labels

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
          idx = idx.tolist()        
        return torch.tensor(self.radar_heartbeat[idx]), self.labels[idx]

    def __len__(self):
        return len(self.radar_heartbeat)


dataset = HCU_Dataset(data_tensor_list, labels_tensor)

データセットでラベルが変わる区切りを発見する

In [65]:
dataset[1000]

/tmp/ipykernel_18531/2876418290.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.radar_heartbeat[idx]), self.labels[idx]


(tensor([[0.0105, 0.0106, 0.0106,  ..., 0.0023, 0.0022, 0.0022]],
        dtype=torch.float64),
 tensor(2))